# Inverse Geometry of the Talos Humanoid Robot

We show an example of inverse geometry using Pinocchio and the SciPy ``minimize`` function

In [ ]:
#setup nice plotting
%matplotlib widget

# we import useful libraries
import time
import numpy as np
import pinocchio
import os
import meshcat
import matplotlib as mp
import matplotlib.pyplot as plt
import scipy

np.set_printoptions(suppress=True, precision=4) # we don't want to print every decimal!

In [ ]:
# the directory where the robot models are located
package_dirs = './urdf/'

# the name of the URDF model (the robot model)
urdf_file = 'talos_reduced.urdf'

# the number of the frames corresponding to limbs of interest (as exposed in Pinocchio)
BASE_LINK_ID = 2
LEFT_FOOT_ID = 16
RIGHT_FOOT_ID = 30
LEFT_HAND_ID = 56
RIGHT_HAND_ID = 90
HEAD_ID = 106

# we load the urdf models with Pinocchio
urdf = package_dirs + urdf_file
robot = pinocchio.RobotWrapper.BuildFromURDF(urdf, package_dirs, root_joint=pinocchio.JointModelFreeFlyer())

# we create the visualization
viz = pinocchio.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)

try:
    viz.initViewer(open=True)
except ImportError as err:
    print("Error while initializing the viewer. It seems you should install Python meshcat")
    print(err)
    sys.exit(0)
    
viz.loadViewerModel()


# place the robot in its default position and display it
q0 = pinocchio.neutral(robot.model)
viz.display(q0)

# create a ball to visualize the goal
viz.viewer['ball'].set_object(meshcat.geometry.Sphere(0.05), 
                              meshcat.geometry.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))

In [ ]:
# this is the desired right hand position (no orientation)
right_hand_desired = np.array([0.4,0.0, 0.1])
viz.viewer['ball'].set_transform(meshcat.transformations.translation_matrix(right_hand_desired))

In [ ]:
def right_hand_error(theta, desired_position):
    # compute the forward kinematics
    robot.forwardKinematics(theta)
    
    # get the position of the end-effector
    current_guess_position = robot.framePlacement(theta, RIGHT_HAND_ID).translation

    error = current_guess_position - desired_position
    return error.dot(error) #+ 0.01 * theta[7:28].dot(theta[7:28])


# our initial guess
q0 = pinocchio.neutral(robot.model)
viz.display(q0)
print(q0)

In [ ]:
res = scipy.optimize.minimize(right_hand_error, q0, args = (right_hand_desired))

robot.forwardKinematics(res.x)
final_pos = robot.framePlacement(res.x, RIGHT_HAND_ID).translation

print(f"the joint angle solution for after the optimization is:\n {res.x}")
print(f"which would lead to an end-effector position of: {final_pos}")
print(f"hence we made an error of {np.linalg.norm(final_pos - right_hand_desired)*1000} millimeters")


#### here we display our solution ####
viz.display(res.x)

## we need to constrain the floating base
We do not want the robot to move so we need to constraint the first 7 DOFs of the configuration. Further, we want to constraint the degrees of freedom of the robot that do not belong to the hand.

In [ ]:
bounds = [(0., 0.)]*q0.size
bounds[0] = (0,0)
bounds[1] = (0,0)
bounds[2] = (0,0)
bounds[3] = (0,0)
bounds[4] = (0,0)
bounds[5] = (0,0)
bounds[6] = (1,1)

for i in range(29,35):
    bounds[i] = (-np.pi, np.pi)

bounds[30] = (-np.pi, 0)

res = scipy.optimize.minimize(right_hand_error, q0, args = (right_hand_desired), bounds=bounds)

robot.forwardKinematics(res.x)
final_pos = robot.framePlacement(res.x, RIGHT_HAND_ID).translation

print(f"the joint angle solution for after the optimization is:\n {res.x}")
print(f"which would lead to an end-effector position of: {final_pos}")
print(f"hence we made an error of {np.linalg.norm(final_pos - right_hand_desired)*1000} millimeters")


#### here we display our solution ####
viz.display(res.x)

# Multi-objective cost - reaching for two objects
We can also optimize for the pose of both hands (or any other frames) by composing the cost functions.

In [ ]:
# this is the desired right hand position (no orientation)
right_hand_desired = np.array([0.4,0.0, 0.1])
viz.viewer['ball'].set_transform(meshcat.transformations.translation_matrix(right_hand_desired))

# create a ball to visualize the goal
left_hand_desired = np.array([0.4,0.4, 0.1])
viz.viewer['ball2'].set_object(meshcat.geometry.Sphere(0.05), 
                              meshcat.geometry.MeshLambertMaterial(
                             color=0x22ffdd,
                             reflectivity=0.8))
viz.viewer['ball2'].set_transform(meshcat.transformations.translation_matrix(left_hand_desired))

q0 = pinocchio.neutral(robot.model)
viz.display(q0)

In [ ]:
def both_hand_error(theta, rh_des, lh_des):
    # compute the forward kinematics
    robot.forwardKinematics(theta)
    
    # get the position of the end-effector
    current_rh_guess_position = robot.framePlacement(theta, RIGHT_HAND_ID).translation
    current_lh_guess_position = robot.framePlacement(theta, LEFT_HAND_ID).translation

    rh_error = current_rh_guess_position - rh_des
    rh_error = rh_error.dot(rh_error)
    lh_error = current_lh_guess_position - lh_des
    lh_error = lh_error.dot(lh_error)
    return lh_error + rh_error

bounds = [(0., 0.)]*q0.size
bounds[0] = (0,0)
bounds[1] = (0,0)
bounds[2] = (0,0)
bounds[3] = (0,0)
bounds[4] = (0,0)
bounds[5] = (0,0)
bounds[6] = (1,1)

# we allow the right hand to move
for i in range(29,35):
    bounds[i] = (-np.pi, np.pi)

# we allos the left hand to move
for i in range(21,27):
    bounds[i] = (-np.pi, np.pi)


bounds[22] = (0, np.pi)
bounds[30] = (-np.pi, 0)

res = scipy.optimize.minimize(both_hand_error, q0, args = (right_hand_desired, left_hand_desired), bounds=bounds)

robot.forwardKinematics(res.x)
rh_final_pos = robot.framePlacement(res.x, RIGHT_HAND_ID).translation
lh_final_pos = robot.framePlacement(res.x, LEFT_HAND_ID).translation

print(f"the joint angle solution for after the optimization is:\n {res.x}")
print(f"which would lead to an end-effector position of RH: {rh_final_pos} and LH: {lh_final_pos}")
print(f"hence we made an error of {np.linalg.norm(final_pos - right_hand_desired)*1000} millimeters")


#### here we display our solution ####
viz.display(res.x)